In [10]:
import networkx as nx
from itertools import product
import numpy as np
import pandas as pd

In [5]:
# Sample edge list
users = [0, 0, 1, 1, 2, 2]
items = [0, 1, 2, 1, 0, 2]
steps = [0, 0, 1, 1, 2, 2]
df = pd.DataFrame({"user_id": users, "item_id": items, "timestamp": steps}).sort_values(by="timestamp")
df

,user_id,item_id,timestamp
0,0,0,0
1,0,1,0
2,1,2,1
3,1,1,1
4,2,0,2
5,2,2,2


In [33]:
user_idx = sorted(df["user_id"].unique())
item_idx = sorted(df["item_id"].unique())

print("{} Users: {} - {}".format(len(user_idx), min(user_idx), max(user_idx)))
print("{} Users: {} - {}".format(len(item_idx), min(item_idx), max(item_idx)))

3 Users: 0 - 2
3 Users: 0 - 2


In [44]:
orig_g = nx.Graph()  # Original bipartite graph
line_g = nx.Graph()  # Line graph of orig_g

def get_user_id(idx):
    return "u{}".format(idx)

def get_item_id(idx):
    return "i{}".format(idx)

user_ids = [get_user_id(n) for n in user_idx]
item_ids = [get_item_id(n) for n in item_idx]
orig_g.add_nodes_from(user_ids + item_ids)

all_possible_edges = list(product(user_ids, item_ids))
line_g.add_nodes_from(all_possible_edges)
nx.set_node_attributes(line_g, 0, "exists")  # Label

In [45]:
prev_ts = 0
for row in df.itertuples(index=False):
    user, item, ts = get_user_id(row.user_id), get_item_id(row.item_id), row.timestamp
    if ts != prev_ts:
        print("==== time {} ====".format(prev_ts))
        print("Line graph edges:", line_g.edges())
        print("Line graph exists:", nx.get_node_attributes(line_g, "exists"))
        prev_ts = ts
    if orig_g.degree(user) > 0:  # This user is already connected
        for i in orig_g.neighbors(user):
            line_g.add_edge((user, item), (user, i))
    if orig_g.degree(item) > 0:  # This item is already connected
        for u in orig_g.neighbors(item):
            line_g.add_edge((user, item), (u, item))
    orig_g.add_edge(user, item, timestamp=ts)
    line_g.nodes[(user, item)]["exists"] = 1

==== time 0 ====
Line graph edges: [(('u0', 'i0'), ('u0', 'i1'))]
Line graph exists: {('u0', 'i0'): 1, ('u0', 'i1'): 1, ('u0', 'i2'): 0, ('u1', 'i0'): 0, ('u1', 'i1'): 0, ('u1', 'i2'): 0, ('u2', 'i0'): 0, ('u2', 'i1'): 0, ('u2', 'i2'): 0}
==== time 1 ====
Line graph edges: [(('u0', 'i0'), ('u0', 'i1')), (('u0', 'i1'), ('u1', 'i1')), (('u1', 'i1'), ('u1', 'i2'))]
Line graph exists: {('u0', 'i0'): 1, ('u0', 'i1'): 1, ('u0', 'i2'): 0, ('u1', 'i0'): 0, ('u1', 'i1'): 1, ('u1', 'i2'): 1, ('u2', 'i0'): 0, ('u2', 'i1'): 0, ('u2', 'i2'): 0}


In [46]:
print("==== time {} ====".format(prev_ts))
print("Line graph edges:", line_g.edges())
print("Line graph exists:", nx.get_node_attributes(line_g, "exists"))

==== time 2 ====
Line graph edges: [(('u0', 'i0'), ('u0', 'i1')), (('u0', 'i0'), ('u2', 'i0')), (('u0', 'i1'), ('u1', 'i1')), (('u1', 'i1'), ('u1', 'i2')), (('u1', 'i2'), ('u2', 'i2')), (('u2', 'i0'), ('u2', 'i2'))]
Line graph exists: {('u0', 'i0'): 1, ('u0', 'i1'): 1, ('u0', 'i2'): 0, ('u1', 'i0'): 0, ('u1', 'i1'): 1, ('u1', 'i2'): 1, ('u2', 'i0'): 1, ('u2', 'i1'): 0, ('u2', 'i2'): 1}
